<a href="https://www.kaggle.com/code/yannicksteph/pca-cv-brain-prediction?scriptVersionId=144348195" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# | PCA | CV | Brain | Prediction |

In [ ]:
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from mpl_toolkits import mplot3d

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, MDS, Isomap, TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import make_scorer, precision_score, recall_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFECV, SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from xgboost.core import DMatrix
import xgboost as xgb

In [ ]:

def get_k(filtered_features):
  return int(len(filtered_features) * 0.75) #3 / 4 )

def plot_components(data, model, images=None, ax=None,
                    thumb_frac=0.05, cmap='gray_r', prefit = False):
    ax = ax or plt.gca()

    if not prefit :
        proj = model.fit_transform(data)
    else:
        proj = data
    ax.plot(proj[:, 0], proj[:, 1], '.b')

    if images is not None:
        min_dist_2 = (thumb_frac * max(proj.max(0) - proj.min(0))) ** 2
        shown_images = np.array([2 * proj.max(0)])
        for i in range(data.shape[0]):
            dist = np.sum((proj[i] - shown_images) ** 2, 1)
            if np.min(dist) < min_dist_2:
                # On ne montre pas le points trop proches
                continue
            shown_images = np.vstack([shown_images, proj[i]])
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(images[i], cmap=cmap),
                                      proj[i])
            ax.add_artist(imagebox)

In [ ]:
dataset_folder_path = "./dataset/"
#dataset_3d_path = "/content/sample_data/3d_rsna_miccai_brain_tumor_brain_segmentation_pytorch_unet.csv"
dataset_3d_path = f"{dataset_folder_path}3d_rsna_miccai_brain_tumor_brain_segmentation_pytorch_unet.csv"

excluded_patient_ids = [109, 123, 709] #, 11, 351, 353, 442, 445, 554, 556, 568, 570, 572, 578, 581, 584, 587, 589, 593, 594, 596, 610]

dataset_df = pd.read_csv(dataset_3d_path)

excluded_features_column_names = ["LeastAxisLength", "Flatness", "gldm_GrayLevelNonUniformityNormalized", "gldm_DependencePercentage"]
dataset_df.drop(excluded_features_column_names, axis=1, inplace=True)

# Patient BraTS21ID now is ID, and ID of Dataset
dataset_df = dataset_df.set_index('ID')
# Drop patient
dataset_df = dataset_df.drop(index=excluded_patient_ids)

In [ ]:

tsne = TSNE(n_components=3)
res = tsne.fit_transform(dataset_df.drop("MGMT_value",axis=1))
plt.figure(figsize=(10,5))
ax = plt.axes(projection='3d')
ax.scatter3D(res[:, 0], res[:, 1], res[:,2], c = dataset_df['MGMT_value'], cmap=plt.cm.Spectral, alpha = .7, s = 4) #**colorize)
plt.axis('equal');


In [ ]:
pd.set_option('display.max_rows', None)

features_selection = False
if (features_selection) :
  #matrice de corrélation
  correlation_matrix = dataset_df.drop("MGMT_value",axis=1).corr()
  #print(correlation_matrix)

  corr=0.75

  highly_correlated_features = set()
  # Identification des paires de caractéristiques fortement corrélées
  for i in range(len(correlation_matrix.columns)):
    for j in range(i):
      if abs(correlation_matrix.iloc[i, j]) > corr:  # Définissez votre seuil de corrélation ici
            colname_i = correlation_matrix.columns[i]
            colname_j = correlation_matrix.columns[j]
            if colname_i not in highly_correlated_features:
                highly_correlated_features.add(colname_j)

  # Suppression des caractéristiques fortement corrélées du jeu de données
  dataset_df = dataset_df.drop(highly_correlated_features, axis=1)

  # Sélection des features obtenues via la bibliothèque radiomics3d
  selected_features = dataset_df.columns

  # Séparation des features et du target
  X = dataset_df[selected_features]
else :
  X = dataset_df.drop("MGMT_value",axis=1)

y = dataset_df['MGMT_value']



Avec PCA


In [ ]:
pca = PCA()
pca.fit(X)
plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
plt.xlabel('Composante principale')
plt.ylabel('Ratio de variance expliquée')
plt.show()

#display(pca.n_components_)
display(pca.score(X))
pca = PCA(n_components=10)

X = pca.fit_transform(X)

In [ ]:
# Division en ensembles de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Utilisation de StratifiedKFold pour préserver les proportions des classes lors de la validation croisée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

'''
Utilisation de la classe StratifiedKFold pour effectuer une validation croisée plus fine tout en préservant
les proportions des classes lors de la division des données.
Utilisation de GridSearchCV avec une grille de paramètres plus spécifique pour trouver les meilleurs
hyperparamètres du modèle RandomForest.
Ajout d'une étape de calibrage du modèle avec CalibratedClassifierCV pour ajuster les seuils de décision
et améliorer la performance de classification.
Ce code ajoute la sélection des caractéristiques en utilisant SelectKBest avec le test F pour choisir les
meilleures caractéristiques. Les caractéristiques fortement corrélées sont identifiées et exclues avant la sélection des meilleures caractéristiques.
SMOTE pour augmenter artificiellement le jeu de données
SelectKBest pour sélection des variables "pertinentes"
'''

In [ ]:


'''
# Augmentation artificielle des données avec SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


# Prétraitement des données : Standardisation des caractéristiques

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Sélection des meilleures caractéristiques avec SelectKBest
k = get_k(selected_features)
selector = SelectKBest(score_func=f_classif, k=k)  # Définissez le nombre de caractéristiques à sélectionner

X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

# Affichage des caractéristiques sélectionnées
print("Features sélectionnées :")
selected_features = [feature for feature, selected in zip(X.columns, selector.get_support()) if selected]
print(selected_features)
'''


*pipeline* MODELE

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Créer un pipeline de prétraitement
preprocessor = Pipeline([
    ('scaler', RobustScaler())
])

# Définir les métriques d'évaluation pour optimiser le réglage du seuil
scoring = {
    'Precision': make_scorer(precision_score),
    'Recall': make_scorer(recall_score),
    'F1': make_scorer(f1_score)
}


# Créer un dictionnaire avec les modèles à tester et leurs grilles de paramètres
models = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'model__C': [0.001,0.01,0.1, 1.0, 10.0],
            'model__penalty': [None, 'l2'],
            ##'random_state' : [42],
            ##'model__threshold': [0.3, 0.4, 0.5, 0.6, 0.7]
        }
    },
'Support Vector Machine': {
    'model': SVC(probability=True),
    'params': {
        'model__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 20.0, 50.0],
        'model__kernel': ['linear', 'rbf','poly','sigmoid'],
        'model__gamma': ['scale', 'auto'],
        'model__degree': [2, 3, 4],
        'model__shrinking': [True, False],
    }
},
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'model__n_estimators': [100, 200],
            'model__max_depth': [3, 5, 7,20],
            'model__min_samples_split': [2, 5, 10],
        }
    },

  # XGBoost
   'XGBoost' : {
      'model' : xgb.XGBClassifier(),
      'params' : {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 6, 7],

      }
  }
}

# Effectuer la recherche des meilleurs hyperparamètres pour chaque modèle
for model_name, model_info in models.items():
    print("Entraînement du modèle:", model_name)

    # Définir le modèle du pipeline
    model_info['model'] = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model_info['model'])
    ])

    # Effectuer la recherche des hyperparamètres optimaux
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=cv, error_score='raise', scoring=scoring, refit='F1')
    grid_search.fit(X_train, y_train)

    # Évaluer le modèle sur l'ensemble de test
    y_pred = grid_search.predict(X_test)


    # Afficher les meilleurs paramètres et scores
    print("Meilleurs paramètres : ", grid_search.best_params_)
    print("Meilleur score F1 : ", grid_search.best_score_)

    # Utiliser le modèle avec les meilleurs paramètres sur les données de test
    best_model = grid_search.best_estimator_
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    # Appliquer un seuil manuellement pour la prédiction
    threshold = 0.5  # Seuil par défaut
    y_pred = (y_pred_proba >= threshold).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print("Accuracy train : ", accuracy_score(y_train, grid_search.predict(X_train)))
    # Afficher les résultats
    print("Meilleurs hyperparamètres:", grid_search.best_params_)
    print("Accuracy test:", accuracy)
    print("Précision:", precision)
    print("Rappel:", recall)
    print("Score F1:", f1)
    print("-------------------------------------")